# ADMM-RNN for stochastic ACOPF

In [1]:
# import all libraries needed downstream
# import os
# import wandb
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch_geometric as pyg
# from torch_geometric.loader import DataLoader
# import torch_geometric.utils as utils
import matplotlib
import networkx as nx
from itertools import combinations
import itertools
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
from torch.utils.data import DataLoader, TensorDataset
# from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
# import seaborn as sns
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error


import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer



# Restoration layer

In [7]:
def create_Balance_Restoration(data,nT=24,Ramp=True):
    
    ngen=len(data['G'])
    nbus = len(data['Bus'])
    Bus=data['Bus']
    G=data['G']
    
    
    #========= variables
    Pg = cp.Variable(shape=(ngen,nT), nonneg=True, name="Pg")

    Obj = cp.Variable(shape=(1),nonneg=True, name="Obj") 
    
    

    
    #========= parameters
    Pd=cp.Parameter(shape=(nT),name='Pd')       
    Pg_pred=cp.Parameter(shape=(ngen,nT),name='Pg_pred')        

    
    
#     A_cost= np.multiply(np.eye(ngen,ngen), data['Gen_data'][:]['a'].to_numpy()) 
    b_cost= data['Gen_data'][:]['b'].to_numpy()
    
    
       
    
    ## ========== Constraints
    constraints=[]
    
    
    for t in range(nT):
#         constraints+=[data['Gen_data'][:]['Pmin'].to_numpy() <= Pg[:,t]]
#         constraints+=[Pg[:,t]<=data['Gen_data'][:]['Pmax'].to_numpy()]


        constraints+=[ sum(Pg[:,t]) == Pd[t] ]
              
        
        if Ramp==True:
            if t!=nT-1:
                constraints+=[ Pg[:,t+1]-Pg[:,t]<=data['Gen_data'][:]['RampUp'].to_numpy() ]
                constraints+=[ Pg[:,t]-Pg[:,t+1]<=data['Gen_data'][:]['RampDn'].to_numpy() ]
                              
                              
    
    Obj =  cp.sum_squares(Pg-Pg_pred) 
    objective = cp.Minimize( Obj )

    problem= cp.Problem(objective, constraints)
        

            
    return {'problem':problem,
           'Pd':Pd, 
            'Pg':Pg,
           'Pg_pred':Pg_pred,
           'variables':[Pg],
           'parameters':[Pd,Pg_pred]}



# Training data in correct shape

In [8]:

def TrainingData_ADMM_RNN_Tseq(networkdata,dataset,Nsamples='all',seed=1):

    """

    Xd shape: [8712, ADMM_iter, Sc, T, B]
    input: [8712, ADMM_iter, Sc, T, B+2*G]

    """

    Bus=networkdata['Bus']
    Gen=networkdata['G']
    Gen_data=networkdata['Gen_data']
    DemandSet=networkdata['Demandset']
    G2B=networkdata['G2B']
    Bus=networkdata['Bus']
    Pd_array=networkdata['Pd_array']
    Nload=len(networkdata['Demandset'])
    Nbus=len(networkdata['Bus'])





    Xd=dataset['Xd']; PgX=dataset['PgX']; Lambda=dataset['Lambda']; PgZ=dataset['PgZ']
    T=Xd.shape[4]
    #all are [8712, ADMM_iter, Sc, G/B, T]


    print(f"Reading dataset ... \n Xd shape is [Samples, ADMM_iter, Sc, B, T]: {Xd.shape}")

    data_len=len(Xd)

    # we find the staring instance of each sample
    # the start can be any point with index < data_len-T
    print(f"{Nsamples} samples requested!")
    if Nsamples=='all':
        Nsamples=data_len




    Xd=Xd.transpose((0,1,2,4,3))  #shape [8712, ADMM_iter, Sc, T, B]
    PgX=PgX.transpose((0,1,2,4,3))
    Lambda=Lambda.transpose((0,1,2,4,3))

    plt.figure(figsize=(3,2))
    plt.hist(Lambda.flatten(),bins=20)
    plt.show()

    # for t in range(1,T):
    #   Lambda[:,:,:,t,:] = 0 #Lambda[:,:,:,0,:]

    print('Lambda is filled with zeros for t>1')
    print('Lambda in 24 hours', Lambda[0,0,0,:,0])
    print('Lambda in ADMM iterations of g1',Lambda[0,:,0,0,0])





    PgZ=PgZ.transpose((0,1,2,4,3))


    input=np.concatenate((Xd,PgX,Lambda),axis=4)
    output=PgZ
    print('input shape: [Samples, ADMM_iter, Sc, T, B+2*G]')
    print('input shape: ',input.shape)
    print('output shape: ',output.shape)

    print("[Samples, ADMM_iter, Sc] should be merged for RNN")

    #TODO! don't merge the dimensions to have test data!


    # input=input.reshape(-1,T,len(Bus)+2*len(Gen))
    # output=output.reshape(-1,T,len(Gen))







    #shuffle the data:
    np.random.seed(seed)
    permutation_index = np.random.permutation(len(input))

    input=input[permutation_index]
    output=output[permutation_index]


    return {
        'input':input,
        'output':output
    }

In [ ]:
# trained_model=result['model']



# torch.save(result['model'].state_dict(),f'/content/gdrive/My Drive/GNN_Project/Runs_results/Model_Weights/rnn_ADMM_{system_size}bus_model_exploration_train0.pth')

# Some Functions!

In [108]:
def Test_ADMMwithML(networkdata,TestDataSet,MLmodel,lamda_std=0.3,ADMM_iter=10,rho=0.1,seed=0):

    """

    Input should be TestDataSet

    The input for ADMM_Stochastic_SCOPF_ML should be [Sc,Bus,T]

    """

    Bus=networkdata['Bus']
    Gen=networkdata['G']


    Xd = TestDataSet['Xd'][:10]                #shape [Samples, Sc, Bus, T]
    Lambda_offline = TestDataSet['Lambda'] # [Samples, ADMM_iter, Sc, G ]

    rk_offline = TestDataSet['rk']

    Time_X_k_offline = TestDataSet['Time_X_k']
    Time_Z_k_offline = TestDataSet['Time_Z_k']
    Time_SO = TestDataSet['Time_SO']


    PgZ_truth = TestDataSet['Pg_SO']       #[Sample,Sc,G,T]

    PgX_ADMM = TestDataSet['PgX_ADMM'][:,:,:]     #shape [Samples,ADMM_iter,G,t0]
    PgZ_ADMM = TestDataSet['PgZ_ADMM']             #shape [Samples, ADMM_iter, Sc,G,24]

    print("Test_ADMMwithML Xd shape [Samples, Sc, Bus, T]: ",Xd.shape)


#     print('Lambda_offline shape [Samples, ADMM_iter, Sc, G ]: ',Lambda_offline.shape)




    Nsamples=Xd.shape[0]

    # ADMM_iter=10

    NumSc=Xd.shape[1]
    T=Xd.shape[3]
    

    # outputs!

    PgX_ML = np.zeros((Nsamples,ADMM_iter,len(Gen)))               #[sample,ADMM_iter,G]

    PgZ_ML = np.zeros((Nsamples,ADMM_iter, NumSc,len(Gen),T)) #[sample,ADMM_iter,Sc,G,T]

    np.random.seed(seed)

    Lambda = np.random.normal(0,lamda_std,size=(Nsamples,ADMM_iter,NumSc,len(Gen)))      #[sample,ADMM_iter,Sc,G]

    rk=np.zeros(shape=(Nsamples, ADMM_iter))




    Time_X_k = np.zeros((Nsamples,ADMM_iter))
    Time_Z_k = np.zeros((Nsamples,ADMM_iter))



    error = np.zeros((Nsamples,ADMM_iter))
    error0 = np.zeros((Nsamples,ADMM_iter))








    modelX = create_XUpdate_Stochastic_ACOPF(networkdata, NumSc=NumSc,nT=T,rho=rho,Ramp=True,Tlink=False)
    


    for i in tqdm(range(Nsamples)):
#     for i in tqdm(range(5)):


        res=ADMM_Stochastic_ACOPF_ML(data=networkdata,MLmodel=MLmodel, modelX=modelX,
                                        All_Sc=True,lambda_initial=Lambda[i,0,:,:],
                                        Ramp=True,DemandInstnace=Xd[i,:,:,:] , #demand instance is [Sc,Bus,T]
                                   k_iter=ADMM_iter,rho=rho,print_result=False)

        PgZ_ML[i]=res['PgZ_k']   #shape [ADMM_iter,SC,G,T]
        PgX_ML[i]=res['PgX_k'][:,:,0]        #shape [ADMM_iter,G,t0]
        Lambda[i] = res['lambda_k']
        rk[i]=res['rk']
        # print(PgX.shape)

        Time_X_k[i] = res['Time_X_k']
        Time_Z_k[i] = res['Time_Z_k']






    for i in range(error.shape[0]):
        for k in range(error.shape[1]):
            error0[i,k] = np.square(np.absolute(PgX_ML[i,k,:] - PgZ_truth[i,0,:,0])).sum()/PgZ_truth[i,0,:,0].size
            error[i,k] = np.square(np.absolute(PgZ_ML[i,k,:,:,:] - PgZ_truth[i,:,:,:])).sum().sum().sum()/PgZ_truth[i,:,:,:].size



    error_offline = np.zeros((Nsamples,PgZ_ADMM.shape[1]))
    error0_offline = np.zeros((Nsamples,PgZ_ADMM.shape[1]))


    for i in range(error_offline.shape[0]):
        for k in range(error_offline.shape[1]):
            error0_offline[i,k] = np.square(np.absolute(PgX_ADMM[i,k,:] - PgZ_truth[i,0,:,0])).sum()/PgZ_truth[i,0,:,0].size
            error_offline[i,k] = np.square(np.absolute(PgZ_ADMM[i,k,:,:,:] - PgZ_truth[i,:,:,:])).sum().sum().sum()/PgZ_truth[i,:,:,:].size 





    #=========================== accuracy error
    plt.figure(figsize=(4,2.5))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 2.5))

    y_min = min(error_offline.mean(axis=0))
    y_max = 1.0#max(error0.mean(axis=0))

    ax2.set_title('ML Eror vs truth')
    ax2.plot(error.mean(axis=0),label='error')
    ax2.plot(error0.mean(axis=0),label='error0')
    ax2.set_yscale('log')
    ax2.set_ylim(y_min, y_max)
    ax2.legend()

    ax1.set_title('ADMM Eror vs truth')
    ax1.plot(error_offline.mean(axis=0),label='error')
    ax1.plot(error0_offline.mean(axis=0),label='error0')
    ax1.set_yscale('log')
    ax1.set_ylim(y_min, y_max)
    ax1.legend()
    plt.show()


    #=========================== rk
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 2.5))
    ax1.plot(rk_offline[:,:ADMM_iter].mean(axis=0),label='rk_offline')
    ax1.plot(rk.mean(axis=0),label='rk_ML')
    ax1.legend()
    ax1.set_yscale('log')
    ax1.set_title('rk test data')

    ax2.plot(rk.mean(axis=0),label='rk ML')
    ax2.set_yscale('log')
    ax2.set_title('rk ML ')
    plt.show()

    #=========================== lambda distribution

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 2.5))

    ax2.set_title('Lambda ML')
    ax2.hist(Lambda[:,:,:,0].flatten(),bins=20)
    ax1.set_title('Lambda ADMM offline')
    ax1.hist(Lambda_offline[:,:,:,0].flatten(),bins=20)
    plt.show()






    return {
        'Xd':Xd,
        'Lambda_ML':Lambda,
        'PgX_ML':PgX_ML,
        'PgZ_ML':PgZ_ML,

        'error0_ML':error0,
        'error_ML':error,
        'error0_offline':error0_offline,
        'error_offline':error_offline,

        'PgZ_truth':PgZ_truth,

        'PgX_ADMM_offline':PgX_ADMM,
        'PgZ_ADMM_offline':PgZ_ADMM,
        'Lambda_offline':Lambda_offline,
        'rk_ML':rk,
        'rk_offline':rk_offline,

        'Time_X_k_ML':Time_X_k,
        'Time_Z_k_ML':Time_Z_k,
        'Time_X_k_offline':Time_X_k_offline,
        'Time_Z_k_offline':Time_Z_k_offline,
        'Time_SO':Time_SO}

In [109]:
#=========================
def ADMM_Stochastic_ACOPF_ML(data,MLmodel,
                           modelX=None,k_iter=10,All_Sc=True,
                        rho=10,rho_decay=3e-1,rho_min=1.0,lambda_initial=None,
                        Ramp=True,DemandInstnace=None,Pg0=None,print_result=False):

     #======  data

    Sbase=data['Sbase']
    Bus=data['Bus']    # for b in Bus
    branch=data['branch']
    Lines=data['Lines']
    L2B=data['L2B']
    Pdemand=data['Pdemand']
    DemandSet=data['Demandset']     # for d in Demand_set
    D2B=data['D2B']                  # for d,b in D2B
    Gen_data=data['Gen_data']
    G=data['G']             #for g in Gset
    G2B=data['G2B']               #for g,b in G2B
#     EndTime=data['EndTime']
    Pd_array=data['Pd_array']
    Qd_array=data['Qd_array']


    #======


    # change here!
    if DemandInstnace is not None:
        Pd_sc_array=DemandInstnace.copy() #shape [Sc,Bus,T]
        EndTime=Pd_sc_array.shape[2]-1
        T=range(0,EndTime+1) #0 to 23

        NumSc=Pd_sc_array.shape[0]
        Senarios=range(0,NumSc)
    else:
        print('No demand input!!')



    k_iter_list=range(k_iter)
    residual={'rk':[],'sk':[]}


    Time_X_k=np.array([]) #(k,)
    Time_Z_k=np.array([]) #(k,)




    ## ======================== Initialization =========================================


#     Multi_OPF() for average scenarios

#     PgX[g]
#     PgZ[sc,g,t] => we pass PgZ[g,0,sc] to X-Update

    PgX=np.zeros((len(G)))
    PgZ=np.zeros((NumSc,len(G),len(T))) # => we pass PgZ[:,g,0] to X-Update

    PgX_k=np.zeros(( k_iter,len(G),len(T))) #shape [ADMM_iter,G,T] we do MP-OPF for better convergence
    PgZ_k = np.zeros((k_iter,NumSc,len(G),len(T))) # => shape [ADMM_iter,Sc,G,T]

    if lambda_initial is None:
        lambda_s=np.random.normal(0,0.1,size=(NumSc,len(G)))
    else:
        lambda_s = lambda_initial

#     lambda_s=np.zeros((NumSc,len(G)))
    lambda_k=np.zeros((k_iter,NumSc,len(G))) #shape [ADMM_iter,Sc,G]



    if modelX is None:
        modelX = create_XUpdate_Stochastic_ACOPF(data, NumSc=NumSc,nT=T,rho=rho,Ramp=True,Tlink=False)
    

    #initialization
    result = Solve_Xupdate_ACOPF(data,modelX,Pd_sc_array,PgZ[:,:,0],lambda_s)
    
#     result=Solve_XUpdate_Stochastic_multi_SCOPF(data,modelX=modelX, cont_list=cont_list,
#                                                 PgZ_s=PgZ[:,:,0],lambda_s=lambda_s,rho=rho,
#                                         Ramp=Ramp,DemandInstnace=Pd_sc_array.mean(axis=0),Pg0=Pg0)


    PgX=result['Pg'][:,0]
    PgZ[:,:,:]=result['Pg']



    ## ======================== ADMM loop =========================================


    start_time = time.time()

    for k in range(k_iter):

#         print('\n===== ADMM iteration ',k)

    #============= X-Update ==============================================

        #save lambda as input!
        lambda_k[k] = lambda_s

        result = Solve_Xupdate_ACOPF(data,modelX,Pd_sc_array,PgZ[:,:,0],lambda_s)
        
        
        
#         Solve_XUpdate_Stochastic_multi_SCOPF(data=data,modelX=modelX,cont_list=cont_list,    #XUpdate_Stochastic_OPF()
#                                     PgZ_s=PgZ[:,:,0],lambda_s=lambda_s,
#                                       rho=rho,Ramp=Ramp,
#                                       DemandInstnace=Pd_sc_array.mean(axis=0),  #Pd_sc_array[0,:,0]
#                                       Pg0=Pg0,print_result=False)

        PgX=result['Pg'][:,0] #we take the first value
        PgXt=result['Pg'] #input for ML model
        PgX_k[k]=result['Pg']

        ex_time=result['time']
        Time_X_k = np.concatenate((Time_X_k,np.array([ex_time])))





    #============= Z-Update ==============================================
        if All_Sc==False:
            for sc in Senarios:
                if print_result==True:
                    print('This option is not used in Ml model')


        elif All_Sc==True:
#             if print_result==True:
#                     print('===== iteration %2s'%(k),end='\r')

            #TODO ML here
            # ML input is [batch,T,B+2G]
            # ML output is [batch,T,G], PgZ is [Sc,G,T]
            # Pd_sc_array is [Sc,Bus,T] -> [SC,T,B]
            # lambda_s is [Sc,G]  -> [Sc,T,G]
            # PgX is [G,T] -> [Sc,T,G]

            MLmodel.to(device)
            MLmodel.eval()     #evaluation mode



            Pd_input = Pd_sc_array.transpose((0,2,1))  # Pd_sc_array is [Sc,Bus,T] -> [SC,T,B]
            # print("Pd_input: ",Pd_input.shape)

            lambda_input = np.zeros((NumSc,len(T),len(G))) # lambda_s is [Sc,G]  -> [Sc,T,G]

            lambda_input[:,0,:] = lambda_s #other times are zeros!
            for t in range(1,len(T)):
              lambda_input[:,t,:] = lambda_input[:,0,:] #repeat itself


            PgX_input = np.zeros((NumSc,len(T),len(G))) # PgX is [G,T] -> [Sc,T,G]
            for sc in range(NumSc):
                PgX_input[sc,:,:] = PgXt.T



            inputt = np.concatenate( (Pd_input,PgX_input,lambda_input),axis=2)

            inputt = torch.Tensor(inputt).to(device)

            # print("input.shape: ",input.shape)


            # ML prediction!
            st=time.time()
            PgZ_pred = MLmodel(inputt)[0].detach()
            et=time.time()

            ex_time=et-st
            Time_Z_k = np.concatenate((Time_Z_k,np.array([ex_time])))

            # print("PgZ_pred shape:", PgZ_pred.shape)


            PgZ=np.array(PgZ_pred.cpu()).transpose((0,2,1)) # [Sc,G,T]

            # dual update
            for g_ind,g in enumerate(G):
                for sc_ind,sc in enumerate(Senarios):
                    lambda_s[sc_ind,g_ind]+=rho*(PgX[g_ind]-PgZ[sc_ind,g_ind,0])




            # end of z-update! ===============================================




        #============= Residuals ==============================================

        # k_update
        PgZ_k[k]=PgZ
#         lambda_k[k]=lambda_s

        rk_now=np.square(np.absolute(PgX - PgZ[:,:,0])).mean()

        residual['rk']+=[rk_now]

        # residual['rk']+= mean_squared_error(np.repeat(PgX,len(Senarios)).reshape(len(Senarios),-1)  ,PgZ[:,:,0])

        rho=np.maximum(rho-rho_decay,rho_min)
        # print('\n lambda[sc,g1] is: ',lambda_s[:,0])




    end_time = time.time()
    ex_time=end_time-start_time  #execution time


    if print_result==True:
        print("\n******* finished!!! ********")
        print("ADMM time: %.3f sec"%ex_time)



        plt.figure(figsize=(5,3))

        plt.plot(range(k_iter), residual['rk']   )

        plt.title('Residuals',fontsize=15)
        plt.xlabel('Iteration k',fontsize=12)
        plt.yscale('log')
    #     plt.grid(ls='--')
        plt.show('Plot1')



    #     plt.grid(ls='--')
        plt.figure(figsize=(5,3))
        for sc in Senarios:
            plt.plot(PgZ[sc,0,:],label=sc)
        plt.plot(PgZ[:,0,:].mean(axis=0), linestyle='dashed' , linewidth=2, color='black' ,label='mean')
        # plt.legend()
        plt.title('G1 for sc')
        plt.show()



    return {
        'PgZ':PgZ, #[Sc,G,T]
        'PgX':PgX,
        'PgX_k':PgX_k, #shape [ADMM_iter,G,T]
        'PgZ_k':PgZ_k, #shape [ADMM_iter,Sc,G,T]
        'lambda_k':lambda_k, #shape [ADMM_iter,Sc,G]
        'rk':residual['rk'], #[ADMM_iter],
        'Time_X_k':Time_X_k,
        'Time_Z_k':Time_Z_k
    }

In [110]:
def create_XUpdate_Stochastic_ACOPF(data,nT=24,NumSc=10,rho=1,   
                           Ramp=True,Tlink=False):
    """This function is used for single case studies and execution time."""
    
    #======  data
    
    Sbase=data['Sbase']
    Bus=data['Bus']    # for b in Bus
    branch=data['branch']
    Lines=data['Lines']
    L2B=data['L2B']
    Pdemand=data['Pdemand']
    DemandSet=data['Demandset']     # for d in Demand_set
    D2B=data['D2B']                  # for d,b in D2B
    Gen_data=data['Gen_data']
    G=data['G']             #for g in Gset
    G2B=data['G2B']               #for g,b in G2B
#     EndTime=data['EndTime']
    Pd_array=data['Pd_array']
    Qd_array=data['Qd_array']
    
    
        


    #======
        
    Senarios=range(0,NumSc)
    T=range(nT)
    EndTime=nT-1
        
    
    #====
    
    model =  ConcreteModel(name='ACOPF')


    model.Pg = Var( G,T,bounds=(0, None) )
    model.Qg = Var( G,T,bounds=(None, None) )
    
    
    model.lambdaP = Param(Senarios,G,initialize=0, mutable=True)
    model.PgZ = Param(Senarios,G,initialize=0, mutable=True)
    
#     model.lambdaQ = Param(Bus,T,initialize=0, mutable=True)
    
    model.Pd = Param(Bus,T,initialize=0, mutable=True)
    model.Qd = Param(Bus,T,initialize=0, mutable=True)
    if Tlink==True:
        model.Pg0 = Param(G,mutable=True)
    
    

    model.V2 = Var(Bus,T,bounds=(0, None),initialize=1 )
    model.L2 = Var(Lines,T,bounds=(0, None) )

    model.Pflow = Var( Lines,T,bounds=(None, None) )
    model.Qflow = Var( Lines,T,bounds=(None, None) )


    model.OF = Var( bounds=(0, None) )

    #initialize
#     if Pg0 != None:
#         for g in G:
#             model.Pg0[g].value=Pg0[g]
#             for t in T:
#                 model.Pg[g,t].value=Pg0[g]



    # equations
    vmin=0.9; vmax=1.1; 



    def eqPbalance(model,b,t):
        return sum( model.Pg[g,t] for g,b in G2B.select('*',b) ) - model.Pd[b,t] \
        == sum(model.Pflow[l,i,j,t] for l,i,j in Lines.select('*',b,'*'))\
    -sum(model.Pflow[l,i,j,t]-branch.loc[(l,i,j)]['r']*model.L2[l,i,j,t]         for l,i,j in Lines.select('*','*',b))
    model.eqPbalance=Constraint(Bus,T,rule=eqPbalance)


    def eqQbalance(model,b,t):
        return sum( model.Qg[g,t] for g,b in G2B.select('*',b) ) - model.Qd[b,t] \
        == sum(model.Qflow[l,i,j,t]      for l,i,j in Lines.select('*',b,'*'))\
    -sum(model.Qflow[l,i,j,t]-branch.loc[(l,i,j)]['x']*model.L2[l,i,j,t]    for l,i,j in Lines.select('*','*',b))\
    #+0.5*branch.loc[(l,i,j)]['b_ij']*model.V2[b,t]
    
    model.eqQbalance=Constraint(Bus,T,rule=eqQbalance)




    def eqSij(model,l,i,j,t):
        return model.Pflow[l,i,j,t]**2+model.Qflow[l,i,j,t]**2 <= model.V2[i,t]*model.L2[l,i,j,t] 
    model.eqSij=Constraint(Lines,T,rule=eqSij)

    #
    def eqSij_V(model,l,i,j,t):
        return model.V2[i,t]-model.V2[j,t] ==\
    -branch.loc[(l,i,j)]['z2']*model.L2[l,i,j,t]\
    +2*(branch.loc[(l,i,j)]['r']*model.Pflow[l,i,j,t] + branch.loc[(l,i,j)]['x']*model.Qflow[l,i,j,t] )
    model.eqSij_V=Constraint(Lines,T,rule=eqSij_V)



#     def eqflow2(model,l,i,j,t):
#             return model.Pflow[l,i,j,t] + model.Pflow[l,j,i,t] >= 0
#     model.eqflow2=Constraint(Lines,T,rule=eqflow2)


    def eqVmax(model,i,t): return model.V2[i,t]<= vmax**2
    model.eqVmax=Constraint(Bus,T,rule=eqVmax)

    def eqVmin(model,i,t): return vmin**2 <= model.V2[i,t]
    model.eqVmin=Constraint(Bus,T,rule=eqVmin)


    def eqPijmax(model,l,i,j,t): 
        return model.Pflow[l,i,j,t]*model.Pflow[l,i,j,t] + model.Qflow[l,i,j,t]*model.Qflow[l,i,j,t]\
        <=branch.loc[(l,i,j)]['limit']**2
    model.eqPijmax=Constraint(Lines,T,rule=eqPijmax)

    def eqPijmin(model,l,i,j,t): 
        return -branch.loc[(l,i,j)]['limit']**2 <=\
        model.Pflow[l,i,j,t]*model.Pflow[l,i,j,t] + model.Qflow[l,i,j,t]*model.Qflow[l,i,j,t]
    model.eqPijmin=Constraint(Lines,T,rule=eqPijmin)



    def eqPgmax(model,g,t): return model.Pg[g,t]<=Gen_data.loc[g]['Pmax']
    model.eqPgmax=Constraint(G,T,rule=eqPgmax)

    def eqPgmin(model,g,t): return Gen_data.loc[g]['Pmin'] <= model.Pg[g,t]
    model.eqPgmin=Constraint(G,T,rule=eqPgmin)

    def eqQgmax(model,g,t): return model.Qg[g,t]<= Gen_data.loc[g]['Qmax']
    model.eqQgmax=Constraint(G,T,rule=eqQgmax)

    def eqQgmin(model,g,t): return Gen_data.loc[g]['Qmin'] <= model.Qg[g,t]
    model.eqQgmin=Constraint(G,T,rule=eqQgmin)

    
    


    if Ramp==True:
        def eqRU(model,g,t):
            if t!=EndTime:
                return model.Pg[g,t+1]-model.Pg[g,t]<=Gen_data.loc[g]['RampUp']  #+model.Rup[g,t+1]
            else:
                return Constraint.Skip
        model.eqRU=Constraint(G,T,rule=eqRU)
        
        def eqRD(model,g,t):
            if t!=EndTime:
                return model.Pg[g,t]-model.Pg[g,t+1]<=Gen_data.loc[g]['RampDn']  
            else:
                return Constraint.Skip
        model.eqRD=Constraint(G,T,rule=eqRD)
        
        if Tlink==True:
            def eqPg0up(model,g):
                return model.Pg[g,T[0]]-model.Pg0[g]<=Gen_data.loc[g]['RampUp'] 
            model.eqPg0up=Constraint(G,rule=eqPg0up)
            
            def eqPg0dn(model,g):
                return model.Pg0[g] - model.Pg[g,T[0]]<=Gen_data.loc[g]['RampDn'] 
            model.eqPg0dn=Constraint(G,rule=eqPg0dn)




    model.eqOF = Constraint( expr = model.OF == sum(Gen_data.loc[g]['b']*model.Pg[g,t]
                                                #+Gen_data.loc[g]['a']*model.Pg[g,t]*model.Pg[g,t]
                                                 + 0.0001*Gen_data.loc[g]['b']*model.Qg[g,t]*model.Qg[g,t] 
                                                    for g in G for t in T)
                    +sum( model.lambdaP[sc,g]*(model.Pg[g,0]-model.PgZ[sc,g]) 
                         + 0.5*rho*(model.Pg[g,0]-model.PgZ[sc,g])**2    for g in G for sc in Senarios) )
    
    
    model.obj = Objective( expr = model.OF , sense=pyo.minimize      )
    
    
    
    
    return model


In [102]:
def Solve_Xupdate_ACOPF(data,modelX,Pd_sc_array,PgZ,lambda_s):
    
    
    Bus=data['Bus']
    G=data['G']
    Pd_array=data['Pd_array']
    Qd_array=data['Qd_array']
    
    nT=Pd_sc_array.shape[2]
    NumSc=Pd_sc_array.shape[0]
    
    
    solver = SolverFactory('ipopt')
    
    Qd_sc_array=Pd_sc_array.copy()
        
    for b_ind in range(len(Bus)):
        if Pd_array[b_ind,0]!=0:
            Qd_sc_array[:,b_ind,:] = Pd_sc_array[:,b_ind,:]*(Qd_array[b_ind,0]/Pd_array[b_ind,0])

    
    
    for b_ind,b in enumerate(Bus):
        for t in range(nT):
            modelX.Pd[b,t].value=Pd_sc_array[:,b_ind,t].mean()
            modelX.Qd[b,t].value=Qd_sc_array[:,b_ind,t].mean()    
    

    for sc in range(NumSc):
        for g_ind,g in enumerate(G):
            modelX.PgZ[sc,g].value = PgZ[sc,g_ind]
            modelX.lambdaP[sc,g].value = lambda_s[sc,g_ind]
    
    
    


    results = solver.solve(modelX,tee=False) 
    
    ex_time = results['Solver'][0]['Time']
    
    
    Pgt_array=np.zeros((len(G),nT))
    for ind,g in enumerate(G):
        for t in range(nT):
            Pgt_array[ind,t]=modelX.Pg[g,t].value
    
    
    return {
        'Pg':Pgt_array,
        'time':ex_time
        }
    

In [ ]:
def run_casestudy(data_x_bus,dataset,T=6,seed=1,epochs=100,trained_model=None):

    Bus=data_x_bus['Bus']
    Gen=data_x_bus['G']


    data=TrainingData_ADMM_RNN_Tseq(data_x_bus,dataset,Nsamples='all')

    Full_x, Full_y = torch.Tensor(data['input']), torch.Tensor(data['output'])
    del data
    print('input imported and deleted!')

    # o_f = Full_y.shape[4]


    # create sizes of training, testing and validation data, 80% as training data, 20% validation
    train_size = int((Full_x.shape[0]//4) * 3)
    val_size = int((Full_x.shape[0]//4) )
    # test_size = int((Full_x.shape[0]//3) / 2)

    # divide X features into training, testing and validation respectively
    train_x = Full_x[:train_size,:,:].reshape(-1,T,len(Bus)+2*len(Gen))
    val_x = Full_x[train_size:train_size+val_size,:,:].reshape(-1,T,len(Bus)+2*len(Gen))
    # test_x = Full_x[train_size+val_size:,:,:].reshape(-1,T,len(Bus)+2*len(Gen))
    del Full_x


    # divide Y output into training, testing and validation respectively
    train_y = Full_y[:train_size,:,:].reshape(-1,T,len(Gen))
    val_y = Full_y[train_size:train_size+val_size,:,:].reshape(-1,T,len(Gen))
    # test_y = Full_y[train_size+val_size:,:,:].reshape(-1,T,len(Gen))
    del Full_y

    print("train_x.shape: ",train_x.shape)
    print("train_y.shape: ",train_y.shape)

    # create training, testing and validation data with Tensor dataset
    train_dataset = TensorDataset(train_x, train_y)
    val_dataset = TensorDataset(val_x, val_y)


    num_samples = train_x.shape[0]
    seq_length = train_x.shape[1]
    input_dim = train_x.shape[2]
    output_dim = train_y.shape[2]

    del train_x, train_y
    del val_x, val_y
    # test_dataset = TensorDataset(test_x, test_y)

    batch_size= 16

    # create iterable data loader object for training, validation and testing data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_layers = 3
    # hidden_feat
    # learning_rate = 0.001



    # instantiate the RNN class object with the parameters above
    torch.manual_seed(seed)
    rnn_model = RNN(input_dim, hidden_feat, output_dim,num_layers,batch_size)
    print(rnn_model)

    # count the number of parameters in this model
    tot_params = 0
    for parameter in rnn_model.parameters():
        layer_ws = 1
        for val in parameter.shape:
            layer_ws*=val
        tot_params += layer_ws
    print(f"Total number of parameters = {tot_params}")


  # param_values = create_hyper_combination()





  # =========================================== Training ============================================================================
    if trained_model is not None:
        rnn_model.load_state_dict(torch.load(trained_model))
        print('\n\n loading trained model weights! \n\n')









    start=time.time()
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    del train_dataset, val_dataset

    optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.0001, weight_decay=5e-5)
    criterion = torch.nn.MSELoss()




    training_losses = []
    validation_losses = []
    for epoch in range(epochs+1):
        print('epoch: ',epoch)
        train_loss = train_rnn(rnn_model, train_loader, optimizer)
        valid_loss = evaluate_rnn(rnn_model, val_loader)
        training_losses.append(train_loss)
        validation_losses.append(valid_loss)
        # wandb.log({"training_loss": train_loss, "validation_loss": valid_loss})

        if epoch % 10 == 0:
            print(f'Epoch: {epoch}')
            print(f'\tTrain Loss: {train_loss:.4f}')
            print(f'\t Val. Loss: {valid_loss:.4f}')



    end=time.time()
    #plt.style.use('seaborn-darkgrid')
    plt.subplots(figsize=(5,3))
    plt.plot([i for i in range(len(training_losses))], training_losses, 'r', label='Training loss', linestyle='solid')
    plt.plot([i for i in range(len(validation_losses))], validation_losses, 'g', label='Validation loss', linestyle='dashed')
    plt.legend()
    plt.title(f'RNN Training and Validation loss for {system_size} Bus system',fontsize = 15)
    plt.xlabel('Epochs',fontsize = 12)
    plt.ylabel('MSE Loss',fontsize = 12)
    #plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/DCOPF_experiment_logs/RNN/RNN_training_and_validation_loss_{system_size}_bus.png',dpi=1200)
    plt.show()
    print('Training time is: %.2f'%(end-start))

    training_losses=np.array(training_losses)
    validation_losses=np.array(validation_losses)








    return {
        'training_losses':training_losses,
        'validation_losses':validation_losses,
        'model':rnn_model
    }






In [62]:
# This is the RNN class
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,batch_size):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.batch_size=batch_size
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.pmax = torch.Tensor(data_x_bus['Gen_data']['Pmax'].values).to(device)
        
        self.NT = 24
        self.NB = len(data_x_bus['Bus'])

    def forward(self, x):
        x.to(device)
        batch_size = x.size(0)
        seq_length = x.size(1)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        out = F.sigmoid(out).to(device)

        out = out * self.pmax
        
        Pg_pred = torch.transpose(out,1,2).to(device) #[batch,G,T]
        xd = torch.transpose(x,1,2)[:,:self.NB,:].sum(axis=1).to(device) #[batch,Bus,T]=>[batch,T]
        
        Pg_r, = layer_balance(xd,Pg_pred) #[batch,G,T]
        
        
        
        Pg_r =  torch.transpose(Pg_r,1,2) #[batch,T,G] 
        Pg_pred = torch.transpose(Pg_pred,1,2) #[batch,T,G] 
        
        
        

        return Pg_r,Pg_pred

In [ ]:
# function for model training , returns loss per epoch after summing loss per batch and dividing by number of batches in data loader object
def train_rnn(model, loader,optimizer,device = device):

    NG = len(data_x_bus['G'])
    ND = system_size
    model.to(device)
    model.train()

    epoch_loss = 0

    for inputs, targets in tqdm(loader):

        inputs = inputs.to(device)  #shape: [batch,T, D+PgX+Lambda]
        targets = targets.to(device)
        optimizer.zero_grad()
        # batch_size = inputs.size(0)

        outputs,Pg_pred = model(inputs)
        
        loss = criterion(outputs, targets) + 0.5*criterion(outputs, Pg_pred)


        loss_t0 = criterion(outputs[:,0,:], targets[:,0,:]) #t0 loss is more important


        # ADMM loss t0
        PgX = inputs[:,0,ND:ND+NG].detach()      #[batch,t0,G] D,G,G
        lambda_s = inputs[:,0,ND+NG:].detach() #[batch,t0,G]
        rho = 0.1

        ADMM_loss = 1e-2 * lambda_s * (outputs[:,0,:]-PgX) + 1e-1 *(rho/2)*torch.square(  outputs[:,0,:]-PgX  )
        ADMM_loss = ADMM_loss.mean()




        loss = loss + 10*loss_t0 + 1*ADMM_loss  #you can change and adjust the loss function for better results. This is a heuristic here



        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss/len(loader)

In [41]:
# function for model validation , returns loss per epoch after summing loss per batch and dividing by number of batches in data loader object
def evaluate_rnn(model, loader, device=device):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode

    epoch_loss = 0

    with torch.no_grad():

        for inputs, targets in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            batch_size = inputs.size(0)
            outputs = model(inputs)[0]
            loss = criterion(outputs, targets)

            epoch_loss += loss.item()

        return epoch_loss / len(loader)

In [43]:
# function to run trained model through test data, save model prediction per batch and true generator outputs to list
def test_rnn(model, loader, device=device):
    model.to(device)
    model.eval() # specifies that the model is in evaluation mode

    epoch_loss = 0
    model_preds = []
    actual_y = []

    with torch.no_grad():

        for inputs, targets in loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            batch_size = inputs.size(0)
            outputs = model(inputs)[0]
            model_preds.append(outputs)
            loss = criterion(outputs, targets)
            actual_y.append(targets)
            epoch_loss += loss.item()

        return epoch_loss / len(loader), model_preds, actual_y

# Run from here!


Importing datasets and some functions

In [ ]:
data_file='IEEE_14_bus_Data_PGLib_ACOPF.xlsx'

multiopf_dataset = 'Dataset_ACOPF_14bus_PGLib_t4_10admm_10sc_0.4lambda_exp5_0.1rho_10samples.npz'
system_size = 14

%run ADMM_ACOPF_Functions.ipynb 


setting up GPU and RAM

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

create the Restoration layer
and read the datasets

In [ ]:
data_x_bus=read_data_ACOPF(File=data_file,DemFactor=1.0,print_data=False)

res=create_Balance_Restoration(data_x_bus,nT=4,Ramp=False)

print('wait for restoration layer construction!')
layer_balance = CvxpyLayer(res['problem'], parameters=res['parameters'], 
                       variables=res['variables'])
print('done!')

# read the dataset again
dataset=np.load(multiopf_dataset)

# Run this code for training!


In [ ]:
criterion = torch.nn.MSELoss()
hidden_feat = 16


NumRuns=1
epochs=1

train_loss_array=np.zeros((NumRuns,epochs+1))
val_loss_array=np.zeros((NumRuns,epochs+1))
Models=[]


for run in range(NumRuns):

    print(f'\n\n\t=============== Run {run} =============== \n\n')
    seed=run

    trained_model=None



    result=run_casestudy(data_x_bus,dataset,T=4,seed=seed,epochs=epochs,trained_model=trained_model)

    train_loss_array[run,:]=result['training_losses']
    val_loss_array[run,:]=result['validation_losses']
    Models.append(result['model'])



    trained_model=result['model']





# ML vs Test Data with Labels

In [ ]:
sys=14
Nsamples=10
rho_off=0.01
rho_ML=1.0
lam_off=0.5
lam_ML=0
NumSc=3
seed=0
ADMM_iter=5
nT=4


TestDataSet = np.load('TestDataset_ACOPF_14bus_4t_3sc_0.01rho_0.4lambda_10samples.npz')




r=Test_ADMMwithML(data_x_bus,TestDataSet,MLmodel=trained_model,lamda_std=lam_ML,ADMM_iter=ADMM_iter,rho=rho_ML,seed=seed)



# ML on Test Data with no Labels

In [ ]:
sys=118
Nsamples=50
rho_ML=1.0
lam_ML=0
NumSc=1000
seed=0


TestDataSet = np.load(r'/content/gdrive/My Drive/GNN_Project/ADMM_Stochastic/TestDataset/TestDataset_NoSolution_118bus_0cont_1000sc_50samples.npz')




MLres=Test_ADMMwithML_NoSolution(data_x_bus,TestDataSet,MLmodel=trained_model,lamda_std=lam_ML,ADMM_iter=10,rho=rho_ML,seed=seed)




# End